**Se deja por cada ejecución 2 salidas en sandbox: una nueva con la fecha de ejecución en la ruta del histórico y se actualiza la foto current**

In [ ]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))
from pyspark.sql import functions as F, DataFrame
import datetime as dt
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
from pyspark.sql.window import Window
import pyspark.sql.types as t
from decimal import Decimal
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import monotonically_increasing_id

from dataproc_sdk.dataproc_sdk_datiopysparksession.datiopysparksession import DatioPysparkSession
datioSparkSession = DatioPysparkSession().get_or_create()
from dataproc_sdk.dataproc_sdk_datiopysparksession import datiopysparksession
dataproc = datiopysparksession.DatioPysparkSession().get_or_create()
from dataproc_sdk.dataproc_sdk_schema.datioschema import DatioSchema
from dataproc_sdk.dataproc_sdk_datiofilesystem.datiofilesystem import DatioFileSystem

In [ ]:
# ENVIRONMENT_VARS_PM
ESCENARIO = '0'
FECHA = '0'

**Por defecto vendrá sin parámetros ya que se carga una foto de datos por cada subida de fichero via launchpad y se tomará la última carga**
- ESCENARIO = '' #'escenario ultima titulizacion'
- FECHA = '' #0:si queremos seleccionar entre varias fechas la mas reciente #'2023-11-7'
    - si la FECHA viene vacia se cogen los registros con la fecha más reciente del fichero launchpad

# Configuracion

### Paths in

In [ ]:
# cuando tengamos path definitivo en launchpad, se cambiará a este
path_launchpad = '/data/sandboxes/dslb/data/Joystick/TITULIZACIONES/limites/launchpad/'#limites_titulizaciones.csv'
#path_master =  /data/master/xpar/data/t_xpar_securitization_limit/gf_odate_date_id/
partition = 'closing_date' #gf_odate_date_id

### Paths out

In [ ]:
path_historico = '/data/sandboxes/dslb/data/Joystick/TITULIZACIONES/limites/historico/'

In [ ]:
path_current = '/data/sandboxes/dslb/data/Joystick/TITULIZACIONES/limites/current/'

In [ ]:
path_constantes = '/data/sandboxes/dslb/data/Joystick/TITULIZACIONES/constantes/'

### Columnas

**tener en cuenta posibles cambios en launchpad entre campos**

In [ ]:
#diccionario con la columnas modeladas en MSA y las columnas de trabajo
dic_columns = {'gf_loaded_limits_list_name':'name_list_desc',
               'gf_loaded_limits_list_date':'limit_date',
               'gf_loaded_limits_list_category_name':'limit_type',
               'gf_first_concept_desc':'concept1_desc',
               'gf_first_concept_id':'concept1_value',
               'gf_second_concept_desc':'concept2_desc',
               'gf_second_concept_id':'concept2_value',
               'gf_assigned_limit_value_per':'limit_value',
               'gf_corporate_loan_ind_type':'corporate_loan_flag',
               'gf_project_finance_ind_type':'project_finance_flag',
               'gf_securz_limit_portfolio_scope_id':'limit_scope',
               'gf_process_active_limit_ind_type':'active_flag',
               'gf_microstrategy_vslztn_order_ind_type':'visual_order',
               'gf_complex_limit_ind_type':'complex_limit'
               }

# Funciones

In [ ]:
# detectar el formato de la fecha en un string
def get_date (fecha:str):
    if(fecha!=None):
        l=re.findall(r'\d+', fecha)
        if(len(l)==3):
            if(len(l[0])==4):
                p_year = l[0]
                p_month = l[1]
                p_day = l[2]
            else:
                p_day = l[0]
                p_month = l[1]
                p_year = l[2]
            return p_day+'-'+p_month+'-'+p_year
        else:
            return None
    else:
        return None
    
# print(get_date('07/11/2023')) #2023-11-07 00:00:00

In [ ]:
# # calculamos la fecha más reciente del dataframe tomando como campo de particion el pasado como parámetro
# def last_partition (p_df:DataFrame, campo:str):
#     window = Window.partitionBy().orderBy(F.col(campo).desc())
    
#     # opcion 1
#     df = p_df.withColumn("rn", F.row_number().over(window)
#                                ).where(F.col("rn") == 1).drop('rn')
    
#     fecha = [x[campo] for x in df.select(campo).collect()][0]
#     print('Fecha de datos:',fecha)
#     return fecha
    

In [ ]:
# calculamos la fecha más reciente de la ruta tomando como campo de particion el pasado como parámetro
def last_partition (p_path:str, campo:str):
    
    datio_path = DatioFileSystem().get().qualify(p_path)
    fs = datio_path.fileSystem()
    path = datio_path.path()
    path_list = fs.listStatus(path)
    paths = [path.getPath().toString() for path in path_list] #listado de todos los paths de la ruta pasada
    
    l_fechas = [element.split(campo+'=')[1] for element in paths if campo in element] #listado de todas las fechas
    return max(l_fechas) # fecha mayor

In [ ]:
# calculamos la fecha más reciente de la ruta tomando como campo de particion el pasado como parámetro
def last_partition_csv (p_path:str, campo:str):    
    datio_path = DatioFileSystem().get().qualify(p_path)
    fs = datio_path.fileSystem()
    path = datio_path.path()
    path_list = fs.listStatus(path)
    paths = [path.getPath().toString() for path in path_list] #listado de todos los paths de la ruta pasada
    
    l_fechas = [(element.split(campo+'=')[1]).split('/')[0] for element in paths if campo in element] #listado de todas las fechas
    return max(l_fechas) # fecha mayor  

In [ ]:
def rename(df,dic):
    for c in dic_columns.keys():
        df = df.withColumnRenamed(c,dic[c])
    return df

In [ ]:
def get_fecha_hoy():
    return format(date.today().strftime("%Y-%m-%d"))

In [ ]:
#borrar columnas con todos los valores nulos
def clean_col_null (df:DataFrame):
    # identify and remove all columns having 100% null values
    df_summary_count = df.summary("count")
    null_cols = [c for c in df_summary_count .columns if df_summary_count.select(c).first()[c] == '0']
    filtered_df = df_summary_count .drop(*null_cols)
    return filtered_df

# Launchpad Limites
información con los límites del inversor

## Seleccionamos la foto a usar
- Si viene marcado por nombre-fecha, tomaremos esa foto
- Si no marcan parámetros se deja la última foto cargada

In [ ]:
# Analizamos si tenemos parámetros de entrada
con_parametros = 0
p_escenario = 0
p_fecha = 0

if((ESCENARIO!= None) & (ESCENARIO!='') & (ESCENARIO!='0')):
    print('Escenario fijado: ' ,ESCENARIO)
    p_escenario= ESCENARIO
    con_parametros=1
    
if ((FECHA!= None) & (FECHA!='') & (FECHA!=0) & (FECHA!='0')):
    print('Fecha fijada: ' ,FECHA)
    p_fecha=FECHA
    con_parametros=1

if(con_parametros==0):
    print('SIN Parametros de Entrada: última foto')
else:
    print('CON Parametros de Entrada: escenario/fecha')

**(*)Modificar por lectura parquet del la ruta master en data lake del launchpad**
path_master =  /data/master/xpar/data/t_xpar_securitization_limit/gf_odate_date_id/

In [ ]:
# Si tenemos parámetros de entrada: leemos todas las fotos cargadas y filtramos
if(con_parametros==1):
    launchpad = spark.read.option('header','True').option('delimiter',';').csv(path_launchpad)
    
    key=['limit_type','concept1_desc','concept1_value','concept2_desc',
         'concept2_value','corporate_loan_flag','project_finance_flag'] # fijamos al nivel que tiene que quedar la tabla

    if((p_escenario!=0) & (p_fecha!=0)): # foto de limites fijada
        launchpad = launchpad.where((F.col('name_list_desc')==ESCENARIO)&(F.col('limit_date')==FECHA))
        print('FOTO fijada: ', ESCENARIO+'_'+FECHA)
    else:
        window = Window.partitionBy(*key).orderBy(F.col("limit_date").desc()) 
        if (p_escenario!=0): # escenario con ultima foto        
            launchpad = launchpad.where(F.col('name_list_desc')==ESCENARIO)
        launchpad = launchpad.withColumn("rn", F.row_number().over(window)).where(F.col("rn") == 1).drop('rn')
        
# ultima foto        
else:
    last_fecha = last_partition_csv (path_launchpad, 'closing_date')
    print('Ultima foto de launchpad: ',last_fecha)
    path_launchpad_f = path_launchpad + 'closing_date=' + str(last_fecha)
    launchpad = spark.read.option('header','True').option('delimiter',';').csv(path_launchpad_f) 

In [ ]:
launchpad.show(5,False)

In [ ]:
limites = rename(launchpad,dic_columns).withColumn('id_limit',monotonically_increasing_id())

In [ ]:
limites.show(2,False)

In [ ]:
limites.where(F.col('limit_type')=='sts_group').show(1,False)

In [ ]:
# tomamos solo los límites activos
limites_in = limites.where(F.col('active_flag')==1) 

In [ ]:
limites_in.select('name_list_desc').distinct().show(5,False)

In [ ]:
# limites.printSchema()

In [ ]:
print('número de limites establecidos:',limites.select('limit_type').distinct().count())
print('número de registros:',limites.count())

## Quitamos posibles duplicados
fijamos la clave por cada registro para evitar duplicidades

In [ ]:
# fijamos al nivel que tiene que quedar la tabla
key=['limit_type','concept1_desc','concept1_value','concept2_desc',
     'concept2_value','corporate_loan_flag','project_finance_flag'] 

In [ ]:
# si hay duplicados nos quedamos con fecha más reciente
if(limites_in.groupBy(*key).agg(F.count('limit_type').alias('n')).where(F.col('n')>1).count()>0):
    print('hay duplicados')
    limites_in.groupBy(*key).agg(F.count('limit_type').alias('n')).where(F.col('n')>1).show(5,False)
    window = Window.partitionBy(*key).orderBy(F.col("limit_date").desc())
    limites_in = limites_in.withColumn("rn", F.row_number().over(window)).where(F.col("rn") == 1).drop('rn')  
else:
    print('no hay duplicados')

### Lista de categorías

In [ ]:
# categorías = [x.limit_type for x in limites.select('limit_type').distinct().collect()]
# sorted(categorías)

In [ ]:
limites_in.groupBy('limit_type','corporate_loan_flag','project_finance_flag').count().orderBy('limit_type').show(500,False)

# Selección Target
Nos quedamos con los límites que aplican según tipologia y sts marcado

### Tipologia
Analizamos si la titulización es de tipo:
- Project Finance
- Corporate Loans
- Ambas

In [ ]:
tipo_titulizacion = limites_in.where(F.col('limit_type')=='portfolio_type').select('corporate_loan_flag','project_finance_flag')
tipo_titulizacion.show()

In [ ]:
corporate_flag = tipo_titulizacion.select('corporate_loan_flag').collect()[0].corporate_loan_flag
project_flag = tipo_titulizacion.select('project_finance_flag').collect()[0].project_finance_flag

In [ ]:
conditions_type = []
if(corporate_flag=='1'):
    print('tipo de titulización Corporate Loan')
    conditions_type.append(F.col('corporate_loan_flag')==1)
elif (project_flag=='1'):
    print('tipo de titulización Project Finance')
    conditions_type.append(F.col('project_finance_flag')==1)
elif ((corporate_flag=='1') & (project_flag=='1')):
    print('tipo de titulización Corporate Loan y Project Finance')
    conditions_type.append((F.col('project_finance_flag')==1)|(F.col('corporate_loan_flag')==1))
else:
    print('ERROR: falta definir la tipología de la titulización')

### STS
Analizamos si se ha marcado que cumpla criterio STS

In [ ]:
marca_sts= 'sts' # nombre del límite que marca si la titulización es de este tipo
condicion = limites_in.where(F.col('limit_type')=='sts'
                            ).select('concept1_desc','concept1_value')
# condicion.show()
sts_flag = condicion.select('concept1_value').collect()[0].concept1_value # valor del limites 'sts'
col_sts = condicion.select('concept1_desc').collect()[0].concept1_desc # columna concepto1 'flag_sts'

if(sts_flag=='1'):
    print ('Titulización CON cumplimiento STS')
    # se deben cumplir los limites asociados a titulizaciones tipo STS     
    limites_current = limites_in.where(((F.col('concept1_value')!=col_sts)|((F.col('concept1_value')==col_sts)&(F.col('concept1_value')==1))))
    if(limites_in.count()!=limites_current.count()):
        print('limites incluidos:')
        limites_current.join(limites_in,['id_limit'],'lefanti').show()
else:
    print ('Titulización SIN cumplimiento STS')
    sts_limites = limites_in.where(((F.col('concept1_value')==col_sts)&(F.col('concept1_value')==1)))
    if(sts_limites.count()>0):
        # quito aquellos limites que solo aplican en titulizaciones tipo sts
        limites_current = limites_in.join(sts_limites,['id_limit'],'lefanti')
    else:# sino hay límites para cumplimiento STS a 1, dejo toda la lista de limites
        limites_current = limites_in

In [ ]:
limites_current.show()

# Persistimos Sbx

### Tipología de columnas
dejamos los tipos ajustados a las necesidades del dato

In [ ]:
cols = limites_current.columns
date_udf = F.udf(get_date)

In [ ]:
# limites_current_f = limites_current.withColumn('limit_date', date_udf(F.col('limit_date'))
#                                               ).withColumn('limit_value', F.col('limit_value').cast('float')
#                                               ).withColumn('corporate_loan_flag', F.col('corporate_loan_flag').cast('int')
#                                               ).withColumn('project_finance_flag', F.col('project_finance_flag').cast('int')
#                                               ).withColumn('active_flag', F.col('active_flag').cast('int')
#                                               ).withColumn('visual_order', F.col('visual_order').cast('int')
#                                               ).select(*cols)

In [ ]:
limites_current_f = limites_current.withColumn('limit_date_p', date_udf('limit_date')
                                              ).withColumn('limit_value_p', F.regexp_replace(F.col('limit_value'),'\.','')
                                              ).withColumn('limit_value', F.regexp_replace(F.col('limit_value_p'),',','\.').cast('float')
                                              ).withColumn('corporate_loan_flag', F.col('corporate_loan_flag').cast('int')
                                              ).withColumn('project_finance_flag', F.col('project_finance_flag').cast('int')
                                              ).withColumn('active_flag', F.col('active_flag').cast('int')
                                              ).withColumn('visual_order', F.col('visual_order').cast('int')
                                              ).withColumn('complex_limit', F.col('complex_limit').cast('int')
                                              ).withColumn('limit_date',F.to_date(F.col("limit_date_p"),'dd-MM-yyyy')
                                              ).select(*cols)

In [ ]:
limites_current_f.printSchema()

In [ ]:
limites_current_f.show(5,False)

## En el histórico
particionado por closing date

In [ ]:
# es la fecha que dejamos para saber que es esa partición del launchpad la que usamos en nuestro limites
print('Partición del launchpad leida',last_fecha)
closing_date = last_fecha

In [ ]:
# si queremos poner la fecha de la ejecución
# closing_date = get_fecha_hoy() # format(date.today().strftime("%Y-%m-%d"))
# print('Fecha de particion: ',closing_date)

In [ ]:
path_hist = path_historico+'closing_date='+closing_date
print(path_hist)

In [ ]:
limites_current_f.write.parquet(path_hist, mode='overwrite')

## Update current

In [ ]:
# dejamos una columna con la fecha del launcpad
limites_current_fc = limites_current_f.withColumn('closing_date', F.lit(closing_date))

In [ ]:
path_current

In [ ]:
limites_current_fc.write.parquet(path_current, mode='overwrite')

## Check

In [ ]:
limites_hist = spark.read.parquet(path_hist)
limites_hist.show(5,False)
limites_hist.select('limit_type').distinct().count()
limites_hist.select('limit_type').distinct().show(500,False)

In [ ]:
limites_current = spark.read.parquet(path_current)
limites_current.show(5,False)
limites_current.select('limit_type').distinct().count()
limites_current.select('limit_type').distinct().show(500,False)